# 

# Azure AI Search: Embeddings

### Load Environment

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

aoai_endpoint = os.getenv('AOAI_ENDPOINT')
aoai_api_key = os.getenv('AOAI_API_KEY')
aoai_api_version = os.getenv('API_VERSION')
aoai_deployment = os.getenv('DEPLOYMENT_NAME')

ai_search_endpoint = os.getenv('AI_SEARCH_ENDPOINT')
ai_search_key = os.getenv('AI_SEARCH_KEY')
ai_search_index_name = os.getenv('AI_SEARCH_INDEX')

### Create Vector Store

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

search_client = SearchClient(
  endpoint=ai_search_endpoint,
  index_name=ai_search_index_name,
  credential=AzureKeyCredential(ai_search_key)
)

In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_deployment=aoai_deployment,
  api_version=aoai_api_version,
  azure_endpoint=aoai_endpoint,
  api_key=aoai_api_key
)

In [ ]:
pages = []

with open("../data/what_is_ai_studio.md", "r") as file:
  from langchain_text_splitters import CharacterTextSplitter

  text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=30)
  pages = text_splitter.split_text(file.read())

In [ ]:
import uuid
documents = []

for item in pages:
  doc = {
    "id": str(uuid.uuid4()),
    "content": item,
    "content_vector": client.embeddings.create(input=item, model=aoai_deployment).data[0].embedding
  }
  documents.append(doc)

result = search_client.upload_documents(documents=documents)